# AWS Virtual Private Cloud

A VPC is a logically isolated networking environment for your AWS resources. In this notebook we'll create a VPC, and partition it into public and private subnets, showing how we can use subnets as security containers or boundaries.

This notebook has been tested in the following regions:

* us-east-1
* us-east-2
* us-west-1
* us-west-2
* ca-central-1
* eu-west-1

## Set Up

We'll use the [AWS Python SDK](https://aws.amazon.com/sdk-for-python/) in this notebook. The first step to using this notebook (after installing it via `pip install boto3`) is to import the boto3 library and create a client. We'll also need the region context for later.

In [ ]:
import boto3

client = boto3.client('ec2')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

## Creating a VPC

In [ ]:
vpc_response = client.create_vpc(
    CidrBlock='10.0.0.0/16',
)

vpcId = vpc_response['Vpc']['VpcId']

print 'created vpc ', vpcId

## Tagging

As you work with AWS, especially under development accounts, you may find at some point you need
clean up unused resources. Human memory being what it is, sometimes it may not be obvious which
resouces are related to which, what can be deleted and what can't, etc.

AWS provides the ability to tag resources with name/value pairs, which is a great way to help out
you future self when it's time to clean up.

The following code shows how to add some helpful tags to the VPC we just created.

In [ ]:
tagged = client.create_tags(
    Resources=[
        vpcId,
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Sample VPC'
        },
        {
            'Key': 'Contact',
            'Value': '@d-smith'
        }
    ]
)

print tagged

Note that many API calls that list resources allow filtering via tags.

In [ ]:
vpcs = client.describe_vpcs(
    Filters=[
        {
            'Name': 'tag:Name',
            'Values': [
                'Sample VPC',
            ]
        },
    ]
)

print vpcs['Vpcs'][0]['VpcId']

## Subnets

Now that we have a VPC, we can create subnets. In contrast to a VPC, which spans all the availability zones in
a region, a subnet is specific to an availability zone.

So the first thing to figure out is what are the availability zones in the region.

In [ ]:
response = client.describe_availability_zones()
availability_zones = response['AvailabilityZones']

for az in availability_zones:
    print az

In [ ]:
# For the rest of this work book we'll create two subnets in two AZs. We'll treat 
# the odds as public, the evens as private.

availability_zones = availability_zones[0:2]
public_subnets = []
private_subnets = []

subnet_no = -1
cidr_base = '10.0.'
for i in range(2):
    subnet_no += 1
    zone_name = availability_zones[i]['ZoneName']
    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )
    
    public_subnets.append(response['Subnet']['SubnetId'])
    

    subnet_no += 1    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )
    
    private_subnets.append(response['Subnet']['SubnetId'])
   
    
print 'Public subnets: ', public_subnets
print 'Private subnets: ', private_subnets

In [ ]:
# We want public addresses on instances launched in the public subnet
# by default.

print public_subnets

for id in public_subnets:
    response = client.modify_subnet_attribute(
        SubnetId=id,
        MapPublicIpOnLaunch={
            'Value': True
        }
    )
    
    print response

## Public Subnets

What makes a subnet public? Having Internet routable IP addresses, and access to the Internet. We've made the
default address option for instances in our public subnets public/routable. To allow Internet access, we create and attach an internet gateway to the VPC, the create a route table with routes from the public subnets to the Internet.

In [ ]:
## Create an internet gateway
igw_response = client.create_internet_gateway()
igwId = igw_response['InternetGateway']['InternetGatewayId']
print 'Created gateway ', igwId

In [ ]:
# Attach internet gateway to VPC
response = client.attach_internet_gateway(
    InternetGatewayId=igw_response['InternetGateway']['InternetGatewayId'],
    VpcId=vpcId
)

print response

Instead of adding an external route to the default route table associated with 
the VPC, we'll create a route table for explicitly controlling subnet access
to the internet. Note that the default route table allows traffic within the 
VPC, and is implicitly associated with the subnets.

In [ ]:
# Create route table for public subnets
rt_response = client.create_route_table(
    VpcId = vpcId
)

routeTableId = rt_response['RouteTable']['RouteTableId']
print 'Create route table ', routeTableId

In [ ]:
# Add route to the interwebs from the gateway

response = client.create_route(
    RouteTableId=routeTableId,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igwId,
)

print response

In [ ]:
# Associate the subnets with the route table
associationIds = []

for subnet in public_subnets:
    response = client.associate_route_table(
        SubnetId=subnet,
        RouteTableId=routeTableId
    )
        
    associationIds.append(response['AssociationId'])

## Private Subnets

Private subnets are subnets that have no ingress from the Internet. There are times, however when access to the internet is needed - for instance when pulling updates from repositories, installing software, accessing AWS services, etc.

In a VPC, a NAT gateway is used to do this. We'll create a NAT gateway for private subnet 1

In [30]:
# First we need a public address for the NAT gateway
response = client.allocate_address(
    Domain='vpc'
)

nat_public_ip = response['PublicIp']
nat_eip_allocation_id = response['AllocationId']
print 'NAT IP:', nat_public_ip, ' NAT Allocation ID: ', nat_eip_allocation_id

NAT IP: 52.60.108.97  NAT Allocation ID:  eipalloc-07fc016e


## Launch an Instance in a Public Subnet

Now that we have a VPC with public subnets, we should be able to launch an instance into a public subnet and
connect to it. To allow connectivity to be verified, we'll use user data to install a LAMP stack with a test
page we can retrieve.

First, we'll need a security group allowing ingres

In [ ]:
#Create security group
response = client.create_security_group(
    GroupName='web_sg',
    Description='Use for launching public web service',
    VpcId=vpcId
)

webSgID = response['GroupId']
print 'created security group ', webSgID

In [ ]:
# Allow ingress on port 80
response = client.authorize_security_group_ingress(
    GroupId=webSgID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response
            

In [ ]:
# User data
user_data = \
"""#!/bin/bash
yum update -y
yum install -y httpd24 php56 mysql55-server php56-mysqlnd
service httpd start
chkconfig httpd on
groupadd www
usermod -a -G www ec2-user
chown -R root:www /var/www
chmod 2775 /var/www
find /var/www -type d -exec chmod 2775 {} +
find /var/www -type f -exec chmod 0664 {} +
echo "<?php phpinfo(); ?>" > /var/www/html/phpinfo.php"""



In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
# Launch instance

response = client.run_instances(
    DryRun=False,
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    SecurityGroupIds=[
        webSgID,
    ],
    UserData=user_data,
    InstanceType='t2.micro',
    SubnetId=public_subnets[0],
    
)

instanceId = response['Instances'][0]['InstanceId']

In [ ]:
# Wait for instance
print 'Waiting for launch of ', instanceId
waiter = client.get_waiter('instance_running')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)

In [ ]:
# Describe the instance - we're interested in the public IP address for the next step
response = client.describe_instances(
    InstanceIds=[
        instanceId,
    ]
)

ipAddress = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
print ipAddress

In [ ]:
# Grab the proxy config from the environment
import os
proxy = os.environ['https_proxy']

proxyParts =  proxy.split(':')
proxyPort = proxyParts[2]
proxyHost = proxyParts[1].split('//')[1]
print proxyHost, proxyPort


In [ ]:
# Do a get on the endpoint
import httplib

def printText(txt):
    lines = txt.split('\n')
    for line in lines:
        print line.strip()

conn = httplib.HTTPConnection(proxyHost, proxyPort)

conn.request('GET', 'http://' + ipAddress + '/')

response = conn.getresponse()
printText (response.read())


## Elastic IPs

In [ ]:
# Create an elastic IP
response = client.allocate_address(
    Domain='vpc'
)

publicIP = response['PublicIp']
allocationID = response['AllocationId']
print 'Elastic IP:', publicIP, ' Allocation ID: ', allocationID

In [ ]:
# Attach this to the instance we created above.
response = client.associate_address(
    InstanceId=instanceId,
    AllocationId=allocationID
)

associationID = response['AssociationId']
print 'Association ID: ',associationID


In [ ]:
# Now do a get on the endpoint using the elastic IP
conn = httplib.HTTPConnection(proxyHost, proxyPort)

conn.request('GET', 'http://' + publicIP + '/')

response = conn.getresponse()
printText (response.read())

In [ ]:
# Clean up

# Disassociate the elastic IP - we will use this later so we do not
# remove it yet.
response = client.disassociate_address(
    AssociationId=associationID
)
print response

In [ ]:

# EC2 Instance
response = client.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)

print response

print 'Waiting for termination to complete'
waiter = client.get_waiter('instance_terminated')
waiter.wait(
   InstanceIds=[
       instanceId,
   ]
)

## Launch an Instance in a Private Subnet

Let's repeat the above, except we'll launch an instance in a private subnet

In [ ]:
response = client.run_instances(
    DryRun=False,
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    SecurityGroupIds=[
        webSgID,
    ],
    UserData=user_data,
    InstanceType='t2.micro',
    SubnetId=private_subnets[0],
    
)

instanceId = response['Instances'][0]['InstanceId']

# Wait for instance
print 'wait for instance to start'
waiter = client.get_waiter('instance_running')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)

# Describe the instance - we're interested in the public IP address for the next step
private_launch_response = client.describe_instances(
    InstanceIds=[
        instanceId,
    ]
)

### What Addresses Does the Instance In the Private Subnet Have?

In [ ]:
instance = private_launch_response['Reservations'][0]['Instances'][0]
print 'public dns name: ', instance['PublicDnsName']
if 'PublicIpAddress' in instance:
    print 'public ip address: ', instance['PublicIpAddress']
print 'private ip address: ', instance['PrivateIpAddress']
print 'private dns name: ', instance['PrivateDnsName']


### What Happens When We Associate a Public Address?

In [ ]:
# Attach this to the instance we created above.
response = client.associate_address(
    InstanceId=instanceId,
    AllocationId=allocationID
)

associationID = response['AssociationId']
print 'Association ID: ',associationID

In [ ]:
private_launch_response = client.describe_instances(
    InstanceIds=[
        instanceId,
    ]
)


In [ ]:
instance = private_launch_response['Reservations'][0]['Instances'][0]
print 'public dns name: ', instance['PublicDnsName']
if 'PublicIpAddress' in instance:
    print 'public ip address: ', instance['PublicIpAddress']
print 'private ip address: ', instance['PrivateIpAddress']
print 'private dns name: ', instance['PrivateDnsName']


### Can We Access the Instance in the Private Subnet Now?

In [ ]:
# Do a get on the endpoint using the elastic IP
conn = httplib.HTTPConnection(proxyHost, proxyPort)

conn.request('GET', 'http://' + publicIP + '/')

response = conn.getresponse()
printText (response.read())

### Let's Add a Route to the Internet Gateway and Try It Again

In [ ]:
response = client.associate_route_table(
        SubnetId=instance['SubnetId'],
        RouteTableId=routeTableId
    )

print response
        
associationIds.append(response['AssociationId'])

In [ ]:
private_launch_response = client.describe_instances(
    InstanceIds=[
        instanceId,
    ]
)

In [ ]:
instance = private_launch_response['Reservations'][0]['Instances'][0]
print instance

print 'public dns name: ', instance['PublicDnsName']
if 'PublicIpAddress' in instance:
    print 'public ip address: ', instance['PublicIpAddress']
print 'private ip address: ', instance['PrivateIpAddress']
print 'private dns name: ', instance['PrivateDnsName']

In [ ]:
# Do a get on the endpoint using the elastic IP
conn = httplib.HTTPConnection(proxyHost, proxyPort)

conn.request('GET', 'http://' + publicIP + '/')

response = conn.getresponse()
printText (response.read())

### Demo 

Using the console and the command line...

* Assign a public address to the instance via an elastic IP address
* Show that the endpoint cannot be reached via curl even when assigned a public IP
* Add the subnet the instance was placed in to the route table allowing ingress
from the internet
* Show the host can now be reached via curl
* Remove the elastic IP address from the instance, and curl again
* Remote the subnet from the route table prior to clean up

In [ ]:
# Clean up
client.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)

waiter = client.get_waiter('instance_terminated')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)



In [ ]:
# Clean up security group
response = client.delete_security_group(
    GroupId=webSgID
)

print response

## EIP Clean Up

In [ ]:
response = client.release_address(
    AllocationId=allocationID
)

print response

## VPC Clean Up

In [31]:
# Release the NAT gateway EIP
response = client.release_address(
    AllocationId=nat_eip_allocation_id
)
print response

{'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '513dbc00-4a57-41e8-b018-a217bfa60b38', 'HTTPHeaders': {'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'server': 'AmazonEC2', 'content-type': 'text/xml;charset=UTF-8', 'date': 'Fri, 10 Mar 2017 22:19:12 GMT'}}}


In [ ]:
# Disassociate the public subnets from the route table
for association in associationIds:
    response = client.disassociate_route_table(
    AssociationId=association
)

In [ ]:
# Delete the route table
response = client.delete_route_table(
    RouteTableId = routeTableId
)

In [ ]:
# Clean up gateway
response = client.detach_internet_gateway(
    InternetGatewayId=igwId,
    VpcId=vpcId
)
print response

response = client.delete_internet_gateway(
    InternetGatewayId=igwId
)
print response

In [ ]:
# Delete subnets
response = client.describe_subnets(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                vpcId,
            ]
        },
    ]
)

subnets = response['Subnets']

for sn in subnets:
    response = client.delete_subnet(
        SubnetId=sn['SubnetId']
    )
    
    print response



In [ ]:
# Delete the VPC
response = client.delete_vpc(
    VpcId = vpcId
)

print response